In [1]:
import torch
import time
import torch.nn.functional as F

In [2]:
def add_new_col_to_inverse(X, old_inverse, new_col, N=None, n=None):
    B = old_inverse
    v = new_col
    u1 = torch.matmul(X.T, v)
    u2 = torch.matmul(B, u1)
    F22inv = 1. / (torch.matmul(v.T, v) - torch.matmul(u1.T, u2))
    u3 = F22inv * u2
    F11inv = B + F22inv * u2 * u2.T
    if N is not None:
        Bnew = torch.cat([
            torch.cat([N - F11inv, -u3], 1),
            torch.cat([-u3.T, n - F22inv], 1)
        ], 0)
    else:
        Bnew = torch.cat([
            torch.cat([F11inv, -u3], 1),
            torch.cat([-u3.T, F22inv], 1)
        ], 0)
    return Bnew

In [44]:
L = 3
U = 6
dim = 3
a = torch.randn(L, dim)
b = torch.randn(U, dim)
b[:3] = a[:3].clone() + F.normalize(torch.randn(3, dim), 1) * .1
# a, b = [F.normalize(i, 1) for i in [a, b]]
m_ab = torch.mm(a, b.T)
m_ba = m_ab.T
m_bb = torch.mm(b, b.T)
m_ba_bb = torch.cat([m_ba, m_bb], 1)
p_ab = torch.softmax(m_ab, 1)

In [45]:
exp_m_ba_bb = torch.exp(m_ba_bb)
exp_m_ba_bb -= exp_m_ba_bb.min()
normaliser = exp_m_ba_bb.sum(1, keepdim=True)

In [46]:
m_bb

tensor([[ 1.3043, -0.7698, -2.4065, -0.7296,  1.9234, -2.7606],
        [-0.7698,  1.6603,  1.4536, -0.6337,  0.1411,  1.6433],
        [-2.4065,  1.4536,  4.8112,  1.0479, -4.8361,  4.6516],
        [-0.7296, -0.6337,  1.0479,  1.5428, -1.2417,  1.8529],
        [ 1.9234,  0.1411, -4.8361, -1.2417,  8.9123, -2.4767],
        [-2.7606,  1.6433,  4.6516,  1.8529, -2.4767,  6.3705]])

In [47]:
Tbar_uu, Tbar_ul = (
    torch.softmax(m_ba_bb, 1)[:, L:], 
    torch.softmax(m_ba_bb, 1)[:, :L]
)
Tbar_uu_unnorm = exp_m_ba_bb[:, L:] - exp_m_ba_bb.min()

In [48]:
N = torch.diag(1 / normaliser[:, 0])
Nm1 = torch.diag(normaliser[:, 0])

In [49]:
middle = torch.inverse(torch.clamp(torch.eye(U) - Tbar_uu, -1, 1 - 1e-8))

In [50]:
vals, V = torch.eig(Tbar_uu, True)

In [54]:
V.numpy().round(3)

array([[-0.495, -0.051,  0.414,  0.701,  0.592,  0.784],
       [-0.061, -0.334, -0.903,  0.158,  0.404,  0.001],
       [-0.004, -0.422,  0.075,  0.016,  0.158, -0.216],
       [-0.04 , -0.517,  0.091, -0.696,  0.676,  0.579],
       [-0.866,  0.001, -0.   , -0.001, -0.001, -0.001],
       [-0.005, -0.664, -0.008,  0.005, -0.065,  0.05 ]], dtype=float32)

In [59]:
middle.numpy().round(1)

array([[1.700e+00, 1.000e-01, 2.000e-01, 1.000e-01, 3.460e+02, 4.000e-01],
       [1.000e-01, 1.300e+00, 7.000e-01, 1.000e-01, 4.260e+01, 1.800e+00],
       [0.000e+00, 1.000e-01, 2.000e+00, 0.000e+00, 2.900e+00, 2.300e+00],
       [1.000e-01, 1.000e-01, 9.000e-01, 1.400e+00, 2.780e+01, 2.800e+00],
       [7.000e-01, 2.000e-01, 1.000e-01, 1.000e-01, 6.062e+02, 4.000e-01],
       [0.000e+00, 1.000e-01, 1.000e+00, 1.000e-01, 3.600e+00, 5.000e+00]],
      dtype=float32)

In [63]:
torch.inverse(V).numpy().round(3)

array([[-1.000e-03, -0.000e+00, -0.000e+00, -0.000e+00, -1.154e+00,
        -1.000e-03],
       [-2.000e-03, -2.100e-02, -3.560e-01, -2.300e-02,  1.300e-02,
        -1.252e+00],
       [ 2.470e-01, -9.030e-01,  1.007e+00,  6.500e-02, -8.400e-02,
        -2.560e-01],
       [ 6.250e-01,  2.370e-01,  3.220e-01, -7.450e-01, -3.430e-01,
         2.070e-01],
       [ 3.040e-01,  3.490e-01,  1.830e+00,  4.160e-01, -2.160e-01,
        -1.688e+00],
       [ 3.560e-01,  0.000e+00, -2.224e+00,  3.160e-01, -2.140e-01,
         1.142e+00]], dtype=float32)

In [52]:
torch.mm(torch.inverse(V), torch.matmul(Tbar_uu, V)).numpy().round(3)

array([[ 0.998, -0.   , -0.   , -0.   , -0.   , -0.   ],
       [-0.   ,  0.824, -0.   ,  0.   , -0.   , -0.   ],
       [-0.   , -0.   ,  0.177, -0.   ,  0.   ,  0.   ],
       [-0.   , -0.   ,  0.   ,  0.21 ,  0.   ,  0.   ],
       [-0.   ,  0.   ,  0.   , -0.   ,  0.289,  0.   ],
       [ 0.   , -0.   , -0.   ,  0.   , -0.   ,  0.254]], dtype=float32)

In [30]:
Tbar_uu.numpy().round(2)

array([[1.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 1.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 1.  , 0.  , 0.  , 0.  , 0.  ],
       [0.13, 0.  , 0.01, 0.19, 0.1 , 0.13],
       [0.  , 0.79, 0.21, 0.  , 0.  , 0.  ],
       [0.01, 0.36, 0.43, 0.02, 0.03, 0.03]], dtype=float32)

In [64]:
p_aba = torch.matmul(torch.matmul(p_ab, middle), Tbar_ul)
p_aba

tensor([[0.6431, 0.1564, 0.2005],
        [0.1126, 0.1315, 0.7560],
        [0.0103, 0.0541, 0.9356]])

In [106]:
p_bb.numpy().round(3)

array([[0.333, 0.   , 0.   , 0.333, 0.   , 0.333],
       [0.   , 0.333, 0.333, 0.   , 0.333, 0.   ],
       [0.333, 0.   , 0.333, 0.   , 0.333, 0.   ],
       [0.333, 0.   , 0.   , 0.   , 0.333, 0.333],
       [0.   , 0.333, 0.333, 0.   , 0.   , 0.333],
       [0.   , 0.333, 0.   , 0.333, 0.   , 0.333]], dtype=float32)

In [107]:
m_bb

tensor([[ 1.3043, -0.7698, -2.4065, -0.7296,  1.9234, -2.7606],
        [-0.7698,  1.6603,  1.4536, -0.6337,  0.1411,  1.6433],
        [-2.4065,  1.4536,  4.8112,  1.0479, -4.8361,  4.6516],
        [-0.7296, -0.6337,  1.0479,  1.5428, -1.2417,  1.8529],
        [ 1.9234,  0.1411, -4.8361, -1.2417,  8.9123, -2.4767],
        [-2.7606,  1.6433,  4.6516,  1.8529, -2.4767,  6.3705]])

In [104]:
p_bb.numpy().round(3)

array([[0.317, 0.04 , 0.008, 0.041, 0.589, 0.005],
       [0.027, 0.312, 0.254, 0.031, 0.068, 0.307],
       [0.   , 0.018, 0.523, 0.012, 0.   , 0.446],
       [0.032, 0.035, 0.187, 0.308, 0.019, 0.419],
       [0.001, 0.   , 0.   , 0.   , 0.999, 0.   ],
       [0.   , 0.007, 0.149, 0.009, 0.   , 0.834]], dtype=float32)

In [98]:
m_bb.argsort(1)

tensor([[5, 2, 1, 3, 0, 4],
        [0, 3, 4, 2, 5, 1],
        [4, 0, 3, 1, 5, 2],
        [4, 0, 1, 2, 3, 5],
        [2, 5, 3, 1, 0, 4],
        [0, 4, 1, 3, 2, 5]])

In [108]:
p_bb

tensor([[0.3333, 0.0000, 0.0000, 0.3333, 0.0000, 0.3333],
        [0.0000, 0.3333, 0.3333, 0.0000, 0.3333, 0.0000],
        [0.3333, 0.0000, 0.3333, 0.0000, 0.3333, 0.0000],
        [0.3333, 0.0000, 0.0000, 0.0000, 0.3333, 0.3333],
        [0.0000, 0.3333, 0.3333, 0.0000, 0.0000, 0.3333],
        [0.0000, 0.3333, 0.0000, 0.3333, 0.0000, 0.3333]])

tensor([0, 3, 5, 1, 2, 4, 0, 2, 4, 0, 4, 5, 1, 2, 5, 1, 3, 5])

In [114]:
[1]

tensor([0, 3, 5, 1, 2, 4, 0, 2, 4, 0, 4, 5, 1, 2, 5, 1, 3, 5])

In [117]:
p_bb

tensor([[0.3333, 0.0000, 0.0000, 0.3333, 0.0000, 0.3333],
        [0.0000, 0.3333, 0.3333, 0.0000, 0.3333, 0.0000],
        [0.3333, 0.0000, 0.3333, 0.0000, 0.3333, 0.0000],
        [0.3333, 0.0000, 0.0000, 0.0000, 0.3333, 0.3333],
        [0.0000, 0.3333, 0.3333, 0.0000, 0.0000, 0.3333],
        [0.0000, 0.3333, 0.0000, 0.3333, 0.0000, 0.3333]])

In [122]:
m_bb[:]

tensor([[ 1.3043, -0.7698, -2.4065, -0.7296,  1.9234, -2.7606],
        [-0.7698,  1.6603,  1.4536, -0.6337,  0.1411,  1.6433],
        [-2.4065,  1.4536,  4.8112,  1.0479, -4.8361,  4.6516],
        [-0.7296, -0.6337,  1.0479,  1.5428, -1.2417,  1.8529],
        [ 1.9234,  0.1411, -4.8361, -1.2417,  8.9123, -2.4767],
        [-2.7606,  1.6433,  4.6516,  1.8529, -2.4767,  6.3705]])

In [119]:
sel

tensor([[ True, False, False,  True, False,  True],
        [False,  True,  True, False,  True, False],
        [ True, False,  True, False,  True, False],
        [ True, False, False, False,  True,  True],
        [False,  True,  True, False, False,  True],
        [False,  True, False,  True, False,  True]])

In [133]:
p_bb

tensor([[0.3333, 0.3333, 0.3333, 0.3333, 0.3333, 0.3333],
        [0.3333, 0.3333, 0.3333, 0.3333, 0.3333, 0.3333],
        [0.3333, 0.3333, 0.3333, 0.3333, 0.3333, 0.3333],
        [0.3333, 0.3333, 0.3333, 0.3333, 0.3333, 0.3333],
        [0.3333, 0.3333, 0.3333, 0.3333, 0.3333, 0.3333],
        [0.3333, 0.3333, 0.3333, 0.3333, 0.3333, 0.3333]])

In [ ]:
torch.

In [143]:
sel[0]

tensor([3, 0, 4])

In [151]:
sel.view(-1)

tensor([[3, 0, 4],
        [2, 5, 1],
        [1, 5, 2],
        [2, 3, 5],
        [1, 0, 4],
        [3, 2, 5]])

In [159]:
sel

tensor([[3, 0, 4],
        [2, 5, 1],
        [1, 5, 2],
        [2, 3, 5],
        [1, 0, 4],
        [3, 2, 5]])

In [165]:
sel

tensor([[3, 0, 4],
        [2, 5, 1],
        [1, 5, 2],
        [2, 3, 5],
        [1, 0, 4],
        [3, 2, 5]])

In [180]:
def get_p_bb_knn(m_bb, n_neighbours=3):
    cols = m_bb.argsort(1)[:, -n_neighbours:]
    p_bb = torch.zeros_like(m_bb)
    put = torch.FloatTensor([1 / n_neighbours])
    rows = torch.arange(p_bb.shape[0]).unsqueeze(1).repeat([1, 3])
    torch.index_put_(p_bb, (rows, sel), put)
    return p_bb

In [215]:
a = torch.arange(3, 0, -1).view(-1, 1).repeat((1, 3)).T
ars = a.argsort(1)[:, -2:]

In [232]:
a[0, :] += 10
a[1, :] += 20
a[2, :] += 30

In [233]:
ars = torch.LongTensor([[1, 0],
        [2, 0],
        [1, 2]])

In [238]:
a[torch.arange(3).unsqueeze(1), ars] = 0.1

In [239]:
a

tensor([[ 0,  0, 11],
        [ 0, 22,  0],
        [33,  0,  0]])

In [218]:
a.index_select(0, ars[0])

tensor([[3, 2, 1],
        [3, 2, 1]])

In [219]:
a

tensor([[3, 2, 1],
        [3, 2, 1],
        [3, 2, 1]])

In [183]:
p_bb = torch.softmax(m_bb, 1)
# p_bb = torch.softmax(m_bb * 0.361, 1)
p_ba = torch.softmax(m_ab.T, 1)
torch.matmul(torch.matmul(p_ab, p_bb), p_ba)

tensor([[0.7408, 0.1587, 0.1005],
        [0.1284, 0.1105, 0.7610],
        [0.0063, 0.0525, 0.9413]])

In [66]:
middle_alternative = torch.mm(
    torch.inverse(Nm1 - Tbar_uu_unnorm), Nm1)

In [181]:
p_bb = get_p_bb_knn(m_bb)
torch.matmul(torch.matmul(p_ab, p_bb), p_ba)

tensor([[0.5351, 0.2124, 0.2525],
        [0.0903, 0.1592, 0.7505],
        [0.0287, 0.1425, 0.8288]])

In [67]:
torch.matmul(torch.matmul(p_ab, middle_alternative), Tbar_ul)

tensor([[0.6466, 0.1568, 0.1984],
        [0.1129, 0.1315, 0.7560],
        [0.0102, 0.0541, 0.9358]])

In [251]:
A = Tbar_uu
# Am1 = torch.inverse(A)
eye = torch.eye(A.shape[0])
# mid = (eye - Am1 + Tbar_uu_unnorm)

In [252]:
torch.inverse(eye - A)

tensor([[1.0100, 0.0100, 0.0100,  ..., 0.0100, 0.0100, 0.0100],
        [0.0100, 1.0100, 0.0100,  ..., 0.0100, 0.0100, 0.0100],
        [0.0100, 0.0100, 1.0100,  ..., 0.0100, 0.0100, 0.0100],
        ...,
        [0.0100, 0.0100, 0.0100,  ..., 1.0100, 0.0100, 0.0100],
        [0.0100, 0.0100, 0.0100,  ..., 0.0100, 1.0100, 0.0100],
        [0.0100, 0.0100, 0.0100,  ..., 0.0100, 0.0100, 1.0100]])

In [232]:
s = eye
for i in range(1, 200):
    s += torch.matrix_power(A, i)

In [233]:
s

tensor([[1.0576, 0.0471, 0.0579,  ..., 0.0476, 0.0434, 0.0583],
        [0.0439, 1.0640, 0.0430,  ..., 0.0576, 0.0502, 0.0460],
        [0.0575, 0.0459, 1.0585,  ..., 0.0462, 0.0434, 0.0572],
        ...,
        [0.0468, 0.0608, 0.0457,  ..., 1.0571, 0.0483, 0.0493],
        [0.0439, 0.0546, 0.0442,  ..., 0.0497, 1.0520, 0.0443],
        [0.0576, 0.0488, 0.0569,  ..., 0.0496, 0.0433, 1.0595]])

In [210]:
eye - torch.inverse(A)

tensor([[ -2361992.7500,   5702304.0000,   4196170.0000,  ...,
            164258.5312,   2823759.7500, -11353244.0000],
        [   595714.0000,  -1052188.5000,   -640998.1250,  ...,
           -131028.6094,   -413537.5938,   2016110.8750],
        [  1999871.1250,  -3800267.7500,  -1949139.6250,  ...,
           -144253.4844,  -1517681.0000,   7819897.0000],
        ...,
        [    50914.8438,   -590212.3750,    -78638.5547,  ...,
           -639337.3750,    256285.2656,    549963.0000],
        [   -53048.3359,   -532025.6250,   -357648.4062,  ...,
           -210970.2969,  -1146699.6250,    634807.3750],
        [  -355720.4688,  -1992918.0000,  -2086678.0000,  ...,
            879219.0625,  -2660131.5000,   3597067.7500]])

In [64]:
X = b.T
XtX = torch.matmul(X.T, X)
# XtX -= XtX.max()
# XtXnew = torch.matmul(X[:, 1:].T, X[:, 1:])
eXtX = torch.exp(XtX)
# eXtXnew = torch.exp(XtXnew)

In [68]:
inv = torch.inverse(Nm1 - Tbar_uu_unnorm)
torch.mm(inv, Nm1)

tensor([[1.6587, 0.4909, 0.6180],
        [0.5769, 1.6031, 0.5878],
        [0.6491, 0.5253, 1.6504]])

In [90]:
torch.eig(Tbar_uu)

torch.return_types.eig(
eigenvalues=tensor([[0.6413, 0.0000],
        [0.0178, 0.0000],
        [0.0963, 0.0000]]),
eigenvectors=tensor([]))

In [74]:
Am1 = torch.inverse(-Tbar_uu_unnorm)
eye = torch.eye(Am1.shape[0])

tensor([[-4.3288, -0.3152,  4.4031],
        [-0.3152, -1.8001,  1.6052],
        [ 4.4031,  1.6052, -6.1487]])

In [ ]:
Am1 - torch.matmul(torch.matmul(Am1, torch.inverse(eye + Am1)), Am1)

In [29]:
X = torch.randn([dim, U]) * 0.3

In [30]:
xnew = torch.randn([dim, 1]) * 0.3
Xnew = torch.cat([X, xnew], 1)
XtXnew = torch.mm(Xnew.T, Xnew)

In [31]:
nm1 = torch.FloatTensor([0.7])
Nm1_new = torch.diag(torch.cat([torch.diagonal(Nm1), nm1], 0))
XtX = torch.mm(X.T, X)

In [32]:
torch.inverse(Nm1 - XtX)

tensor([[ 0.2041, -0.0121, -0.0103],
        [-0.0121,  0.1979,  0.0036],
        [-0.0103,  0.0036,  0.2216]])

In [21]:
# X = torch.randn([128, 4096]) * 0.1
X = torch.randn([5, 4])
# X = F.normalize(X)
XtX = torch.matmul(X.T, X)
# XtX -= XtX.max()
# XtXnew = torch.matmul(X[:, 1:].T, X[:, 1:])
eXtX = torch.exp(XtX)
# eXtXnew = torch.exp(XtXnew)

In [27]:
norm = eXtX.sum(1)
Nm1 = torch.diag(1 / norm)
eye = torch.eye(eXtX.shape[0])

In [28]:
sm = F.softmax(XtX, 1)
sm

tensor([[0.9107, 0.0067, 0.0717, 0.0109],
        [0.0420, 0.6720, 0.0537, 0.2323],
        [0.0689, 0.0082, 0.8986, 0.0242],
        [0.0401, 0.1365, 0.0930, 0.7304]])

In [29]:
torch.matmul(Nm1, eXtX)

tensor([[0.9107, 0.0067, 0.0717, 0.0109],
        [0.0420, 0.6720, 0.0537, 0.2323],
        [0.0689, 0.0082, 0.8986, 0.0242],
        [0.0401, 0.1365, 0.0930, 0.7304]])

In [30]:
torch.inverse(eye - sm)

tensor([[9364086.0000, 1491401.5000, 9740496.0000, 2538917.7500],
        [9364082.0000, 1491406.3750, 9740500.0000, 2538922.2500],
        [9364083.0000, 1491402.1250, 9740505.0000, 2538919.0000],
        [9364084.0000, 1491404.2500, 9740502.0000, 2538924.2500]])

In [274]:
# (I - Nm1.eXtX)-1
# [Nm1 (N - eXtX)]-1
# (N - eXtX)-1 N

In [300]:
vals, vecs = torch.symeig(-eXtX, True)
D = torch.diag(vals)
D

tensor([[-9.7298e+08,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00, -4.1769e+08,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00, -1.0133e+08,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        ...,
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -7.0361e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
         -5.1879e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00, -4.0554e+00]])

In [290]:
torch.mm(u, torch.mm(torch.diag(1 / norm), u.T))

tensor([[ 2.8135e-01,  1.5299e-04,  6.4665e-03,  ...,  6.9279e-04,
          1.9238e-03,  1.7885e-02],
        [ 1.5299e-04,  5.0180e-01,  2.9736e-04,  ...,  1.2506e-03,
          2.4892e-06,  9.5986e-06],
        [ 6.4665e-03,  2.9736e-04,  1.6346e-01,  ...,  2.9574e-02,
          2.3229e-04,  5.7907e-04],
        ...,
        [ 6.9279e-04,  1.2506e-03,  2.9574e-02,  ...,  9.9618e-01,
         -7.1638e-03, -2.8588e-02],
        [ 1.9238e-03,  2.4892e-06,  2.3229e-04,  ..., -7.1638e-03,
          3.4938e-02,  5.9022e-02],
        [ 1.7885e-02,  9.5986e-06,  5.7907e-04,  ..., -2.8588e-02,
          5.9022e-02,  1.3684e+00]])

In [283]:
torch.diag(norm) - eXtX

tensor([[ 3.1511e+02, -6.7083e-02, -2.8354e+00,  ..., -3.0378e-01,
         -8.4356e-01, -7.8423e+00],
        [-6.7083e-02,  8.6006e+03, -5.0743e+00,  ..., -2.1584e+01,
         -2.5371e-02, -2.0796e-03],
        [-2.8354e+00, -5.0743e+00,  1.8221e+03,  ..., -6.4391e+01,
         -4.2899e-01, -5.4613e-01],
        ...,
        [-3.0378e-01, -2.1584e+01, -6.4391e+01,  ...,  3.6427e+03,
         -7.0198e-01, -5.2066e-01],
        [-8.4356e-01, -2.5371e-02, -4.2899e-01,  ..., -7.0198e-01,
          8.3952e+02, -2.8874e+01],
        [-7.8423e+00, -2.0796e-03, -5.4613e-01,  ..., -5.2066e-01,
         -2.8874e+01,  2.6388e+03]])

In [267]:
x = torch.randn(3, 3) * .5
x = torch.mm(x, x.T)
x
upd = x.clone()
upd += torch.randn(3, 3) * .5
upd[:2, :2] = 0
upd += x
vals, vecs = torch.symeig(upd, True)
vals, vecs

(tensor([0.0923, 0.2388, 0.9513]), tensor([[ 0.6054,  0.2232,  0.7640],
         [ 0.5513, -0.8100, -0.2001],
         [-0.5742, -0.5423,  0.6134]]))

In [268]:
upd

tensor([[ 0.6010, -0.1578,  0.3848],
        [-0.1578,  0.2228, -0.0411],
        [ 0.7059, -0.8007,  0.4586]])

In [269]:
x

tensor([[ 0.6010, -0.1578,  0.3630],
        [-0.1578,  0.2228, -0.0642],
        [ 0.3630, -0.0642,  0.3687]])

torch.return_types.symeig(
eigenvalues=tensor([-1.0363,  0.0000,  2.8425]),
eigenvectors=tensor([[ 0.7519,  0.4780,  0.4540],
        [ 0.4092, -0.8784,  0.2471],
        [-0.5169,  0.0000,  0.8561]]))

In [194]:
torch.mm(torch.cholesky_inverse(u), torch.diag(norm))

tensor([[2.9064e-10, 1.5353e-09, 3.0224e-09,  ..., 3.4884e-02, 1.3356e-10,
         1.0679e-10],
        [2.9064e-10, 1.5353e-09, 3.0224e-09,  ..., 3.4884e-02, 1.3356e-10,
         1.0679e-10],
        [2.9064e-10, 1.5353e-09, 3.0224e-09,  ..., 3.4884e-02, 1.3356e-10,
         1.0679e-10],
        ...,
        [2.9064e-10, 1.5353e-09, 3.0224e-09,  ..., 3.4884e-02, 1.3356e-10,
         1.0679e-10],
        [2.9064e-10, 1.5353e-09, 3.0224e-09,  ..., 3.4884e-02, 1.3356e-10,
         1.0679e-10],
        [2.9064e-10, 1.5353e-09, 3.0224e-09,  ..., 3.4884e-02, 1.3356e-10,
         1.0679e-10]])

In [195]:
torch.mm(torch.inverse(torch.diag(norm) - eXtX), torch.diag(norm))

tensor([[ 6.6648e+01,  3.6682e+00,  1.3535e+00,  ..., -2.2834e+07,
          2.2598e-01,  1.6173e-01],
        [ 6.9440e-01,  4.9179e+01,  2.3434e+00,  ..., -1.0266e+07,
          7.6708e-02,  1.2459e-01],
        [ 1.3015e-01,  1.1904e+00,  1.0524e+01,  ..., -7.9501e+05,
          6.6554e-04,  4.6121e-02],
        ...,
        [-1.9025e-01, -4.5183e-01, -6.8883e-02,  ..., -8.7778e+07,
         -5.0916e-02, -1.0791e-02],
        [ 4.9178e-01,  8.8182e-01,  1.5061e-02,  ..., -1.3299e+07,
          1.0252e+01,  2.1782e-01],
        [ 4.4019e-01,  1.7913e+00,  1.3054e+00,  ..., -3.5251e+06,
          2.7242e-01,  1.0420e+01]])

In [196]:
torch.inverse(torch.eye(XtX.shape[0]) - sm)

tensor([[ 6.6366e+01,  1.1420e+00, -8.0439e-01,  ..., -1.9514e+07,
          1.9894e-01,  4.2459e-02],
        [ 2.1640e-01,  4.1305e+01, -2.2012e+00,  ..., -5.8928e+06,
          3.7290e-02, -1.1955e-01],
        [-7.7326e-02, -1.1181e+00,  7.2707e+00,  ...,  5.9601e+05,
         -1.3647e-02, -7.7625e-02],
        ...,
        [-1.6257e-01, -2.5920e-01,  5.2834e-02,  ..., -8.8705e+07,
         -4.6960e-02, -3.7132e-03],
        [ 4.3294e-01,  4.2712e-01, -3.0942e-01,  ..., -1.2264e+07,
          1.0245e+01,  1.9772e-01],
        [ 1.1565e-01, -1.7205e+00, -2.2010e+00,  ..., -1.2067e+06,
          2.4753e-01,  1.0230e+01]])

In [197]:
torch.inverse(torch.eye(XtX.shape[0]) - torch.mm(Nm1, eXtX))

tensor([[ 6.6891e+01,  6.5958e+00,  3.6262e+00,  ..., -2.4819e+07,
          2.4556e-01,  3.0048e-01],
        [ 1.2481e+00,  5.6533e+01,  7.6646e+00,  ..., -1.4582e+07,
          1.1998e-01,  4.4421e-01],
        [ 3.4845e-01,  3.8937e+00,  1.1802e+01,  ..., -2.4234e+06,
          1.8712e-02,  1.8004e-01],
        ...,
        [-2.0666e-01, -6.4044e-01, -2.0948e-01,  ..., -8.7556e+07,
         -5.2416e-02, -1.9330e-02],
        [ 5.3444e-01,  1.3834e+00,  4.2232e-01,  ..., -1.3693e+07,
          1.0256e+01,  2.4122e-01],
        [ 8.1839e-01,  6.3723e+00,  5.0757e+00,  ..., -6.3342e+06,
          3.0154e-01,  1.0645e+01]])

torch.Size([50, 50])

In [24]:
def torch_inv():
    return torch.inverse(eXtX)

%timeit torch_inv()

579 ms ± 7.23 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [31]:
def torch_inv2():
    u = torch.cholesky(eXtX)
    return torch.cholesky_inverse(u)
%timeit torch_inv2()

470 ms ± 20.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [34]:
%timeit u = torch.cholesky(eXtX)

180 ms ± 7.18 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [36]:
torch.mm(u, u.T)

tensor([[0.0413, 0.0183, 0.0127,  ..., 0.0157, 0.0157, 0.0126],
        [0.0183, 0.0516, 0.0125,  ..., 0.0111, 0.0166, 0.0144],
        [0.0127, 0.0125, 0.0454,  ..., 0.0168, 0.0142, 0.0184],
        ...,
        [0.0157, 0.0111, 0.0168,  ..., 0.0465, 0.0145, 0.0170],
        [0.0157, 0.0166, 0.0142,  ..., 0.0145, 0.0436, 0.0137],
        [0.0126, 0.0144, 0.0184,  ..., 0.0170, 0.0137, 0.0439]])

In [26]:

torch.potrf(eXtX)

AttributeError: module 'torch' has no attribute 'potrf'

In [25]:
import scipy.linalg

M = eXtX.numpy()
def scipy_inv(M):
    return np.linalg.inv(M)
    zz , _ = scipy.linalg.lapack.dpotrf(M, False, False)
    inv_M , info = scipy.linalg.lapack.dpotri(zz)
    return inv_M
%timeit scipy_inv(M)

2.94 s ± 90.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [21]:
inv_M

array([[ 5.16016609e+03, -7.17094981e+01,  3.67227918e+01, ...,
        -1.39174757e+01,  9.63197227e+00,  3.24160249e+01],
       [ 1.82854533e-02,  2.65514458e+03, -9.94787787e+00, ...,
        -3.58465863e+01,  1.19178624e+01,  2.36879707e+01],
       [ 1.26858354e-02,  1.24610066e-02,  3.53307662e+03, ...,
         8.30143124e+01,  5.84684897e+01, -1.16068985e+02],
       ...,
       [ 1.57354474e-02,  1.11050010e-02,  1.68003440e-02, ...,
         3.70626661e+03,  1.43761181e+01,  2.37045821e+01],
       [ 1.57182813e-02,  1.65951848e-02,  1.41780972e-02, ...,
         1.44798160e-02,  4.38297233e+03,  2.60644095e+01],
       [ 1.25547051e-02,  1.44284368e-02,  1.84476972e-02, ...,
         1.70407891e-02,  1.36840343e-02,  4.21725737e+03]])

In [22]:
import numpy as np

np.linalg.cholesky(M)

array([[ 2.0313248e-01,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 9.0017378e-02,  2.0856513e-01,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 6.2451042e-02,  3.2792285e-02,  2.0104480e-01, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       ...,
       [ 7.7463962e-02,  1.9811068e-02,  5.6270979e-02, ...,
         1.6426392e-02,  0.0000000e+00,  0.0000000e+00],
       [ 7.7379458e-02,  4.6171136e-02,  3.8954563e-02, ...,
        -5.3331343e-05,  1.5105100e-02,  0.0000000e+00],
       [ 6.1805502e-02,  4.2504072e-02,  6.5627530e-02, ...,
        -9.2000715e-05, -9.3355819e-05,  1.5398731e-02]], dtype=float32)

In [29]:
inv_M

array([[ 3.51934044e+22,  5.62912448e+22,  3.32529420e+22, ...,
        -1.55015399e+05,  1.90540316e+05, -6.83050750e+05],
       [-7.57587026e-04,  9.00368835e+22,  5.31875047e+22, ...,
        -2.48000540e+05,  3.04861960e+05, -1.09240252e+06],
       [ 1.96077349e-03,  6.69133267e-04,  3.14194748e+22, ...,
        -1.46437006e+05,  1.79968692e+05, -6.45486137e+05],
       ...,
       [-1.49789630e-04, -3.62291816e-03,  6.04315009e-03, ...,
         7.83422954e+02, -3.26315742e+01, -1.37612101e+02],
       [-1.92725344e-03, -3.70545220e-03, -4.79730405e-03, ...,
         1.21067581e-03,  8.82375437e+02,  2.54072718e+01],
       [-1.81561953e-03, -4.84478893e-03, -1.69411406e-03, ...,
         4.36846633e-03, -7.56252208e-04,  1.13137123e+03]])

In [31]:
zz

array([[ 1.74134039e-01, -4.35059699e-03,  1.12601390e-02, ...,
        -1.49789630e-04, -1.92725344e-03, -1.81561953e-03],
       [-7.57587026e-04,  1.72435779e-01,  4.16457418e-03, ...,
        -3.62291816e-03, -3.70545220e-03, -4.84478893e-03],
       [ 1.96077349e-03,  6.69133267e-04,  1.72395117e-01, ...,
         6.04315009e-03, -4.79730405e-03, -1.69411406e-03],
       ...,
       [-1.49789630e-04, -3.62291816e-03,  6.04315009e-03, ...,
         3.61386612e-02,  1.21067581e-03,  4.36846633e-03],
       [-1.92725344e-03, -3.70545220e-03, -4.79730405e-03, ...,
         1.21067581e-03,  3.36754769e-02, -7.56252208e-04],
       [-1.81561953e-03, -4.84478893e-03, -1.69411406e-03, ...,
         4.36846633e-03, -7.56252208e-04,  2.97301728e-02]])

In [258]:
X = torch.randn([4096, 128]) * 0.05
def old(X):
    return torch.inverse(torch.exp(torch.matmul(X, X.T)))

%timeit old(X)

705 ms ± 76 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [259]:
old(X).shape

torch.Size([4096, 4096])

In [ ]:
def new(X):
    
    symeig = torch.symeig(torch.exp(torch.matmul(X, X.T)), True)

In [260]:
torch.symeig(torch.exp(torch.matmul(X, X.T)), True)

torch.return_types.symeig(
eigenvalues=tensor([8.9242e-03, 9.1426e-03, 9.1826e-03,  ..., 1.4072e+01, 1.4283e+01,
        4.0980e+03]),
eigenvectors=tensor([[-1.2142e-02,  1.0688e-02, -4.1807e-03,  ...,  1.0539e-02,
          1.1228e-02,  1.5622e-02],
        [ 5.4051e-03, -1.8964e-03,  3.7369e-03,  ...,  1.0931e-02,
         -2.3277e-02,  1.5624e-02],
        [ 3.8326e-03, -6.3312e-03, -7.8157e-03,  ..., -2.2687e-02,
         -2.0749e-02,  1.5607e-02],
        ...,
        [ 3.8040e-03, -2.9918e-03, -8.4959e-03,  ...,  3.2905e-04,
          1.9703e-02,  1.5632e-02],
        [-2.8663e-03, -1.0480e-03,  7.3053e-03,  ..., -6.7421e-03,
         -2.5815e-02,  1.5624e-02],
        [ 1.3776e-02,  7.8041e-03, -7.0944e-05,  ..., -1.7609e-02,
         -1.6254e-02,  1.5626e-02]]))

In [261]:
%timeit torch.symeig(torch.exp(torch.matmul(X, X.T)), True)

10.9 s ± 314 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
torch.inverse()

In [ ]:
c = 1.0 / (1.0 + torch.mm(u.T, torch.mm(old_inv, v.T)))
new_inv = 

In [245]:
u.shape

torch.Size([128])

In [209]:
eXtX

tensor([[1.0224, 0.9655, 1.0029,  ..., 0.9941, 1.0127, 0.9819],
        [0.9655, 1.0879, 1.0113,  ..., 1.0133, 0.9849, 1.0195],
        [1.0029, 1.0113, 1.0152,  ..., 0.9897, 1.0153, 0.9953],
        ...,
        [0.9941, 1.0133, 0.9897,  ..., 1.0278, 0.9684, 0.9949],
        [1.0127, 0.9849, 1.0153,  ..., 0.9684, 1.0607, 1.0039],
        [0.9819, 1.0195, 0.9953,  ..., 0.9949, 1.0039, 1.0234]])

In [210]:
vals, vecs = torch.symeig(eXtX, True)

In [213]:
XtX0 = torch.matmul(X[:, :-1].T, X[:, :-1])
eXtX0 = torch.exp(XtX0)
vals0, vecs0 = torch.symeig(eXtX0, True)

In [193]:
goal = torch.inverse(torch.mm(torch.mm(vecs, torch.diag(vals)), vecs.T))
goal[-1, -1]

tensor(2566537.)

In [194]:
known = torch.inverse(torch.mm(torch.mm(vecs0, torch.diag(vals0)), vecs0.T))

In [ ]:
v

In [ ]:
# %timeit torch.inverse(eXtX)

In [116]:
torch.all(XtX == XtX.T)

tensor(True)

In [132]:
torch.mm(torch.mm(vecs, torch.diag(vals)), vecs.T)

tensor([[1.0348, 0.9936, 0.9993,  ..., 1.0011, 0.9995, 0.9995],
        [0.9936, 1.0373, 0.9996,  ..., 1.0034, 0.9957, 1.0048],
        [0.9993, 0.9996, 1.0306,  ..., 1.0016, 0.9989, 1.0001],
        ...,
        [1.0011, 1.0034, 1.0016,  ..., 1.0285, 0.9987, 0.9958],
        [0.9995, 0.9957, 0.9989,  ..., 0.9987, 1.0331, 0.9967],
        [0.9995, 1.0048, 1.0001,  ..., 0.9958, 0.9967, 1.0325]])

In [137]:
torch.mm(torch.mm(vecs, torch.diag(vals)), vecs.T)

tensor([[1.0348, 0.9936, 0.9993,  ..., 1.0011, 0.9995, 0.9995],
        [0.9936, 1.0373, 0.9996,  ..., 1.0034, 0.9957, 1.0048],
        [0.9993, 0.9996, 1.0306,  ..., 1.0016, 0.9989, 1.0001],
        ...,
        [1.0011, 1.0034, 1.0016,  ..., 1.0285, 0.9987, 0.9958],
        [0.9995, 0.9957, 0.9989,  ..., 0.9987, 1.0331, 0.9967],
        [0.9995, 1.0048, 1.0001,  ..., 0.9958, 0.9967, 1.0325]])

In [50]:
f = torch.exp(XtX - XtX.min())
f / f.sum(1, keepdim=True)

tensor([[0.5380, 0.2225, 0.2395],
        [0.2297, 0.4388, 0.3315],
        [0.2511, 0.3366, 0.4123]])

In [51]:
XtX - XtX.min()

tensor([[0.8831, 0.0000, 0.0738],
        [0.0000, 0.6472, 0.3668],
        [0.0738, 0.3668, 0.5696]])

In [49]:
torch.softmax(XtX, 1)

tensor([[0.5380, 0.2225, 0.2395],
        [0.2297, 0.4388, 0.3315],
        [0.2511, 0.3366, 0.4123]])

In [46]:
torch.inverse(torch.exp(XtX))

tensor([[ 0.7726, -0.1278, -0.3662],
        [-0.1278,  1.8606, -1.4413],
        [-0.3662, -1.4413,  2.1615]])

In [37]:
(torch.inverse(XtX))

tensor([[4.3831, 3.1722, 2.7911],
        [3.1722, 5.3071, 1.2520],
        [2.7911, 1.2520, 5.6468]])

In [152]:
def add_new_col_to_inverse_new(X, old_inverse, new_col, N=None, n=None):
    B = old_inverse
    v = new_col
    # Decomposition of diagonal matrix N
    Nlong = torch.cat([torch.sqrt(N), torch.zeros([X.shape[0] - N.shape[0], N.shape[1]])], 0)
    X = Nlong - X
    
    
    # d = 1. / (torch.matmul(v.T, v) - torch.matmul(u1.T, u2))
    middle = torch.matmul(torch.matmul(Nlong + X, (B)), (Nlong + X).T)
#     print(middle.shape, N.shape, B.shape)
    v = v + n
    other = torch.matmul(torch.matmul(v.T, middle), v)
    d = 1. / (torch.matmul(v.T, v) - other)
    return d
    u3 = d * u2
    F11inv = B + d * u2 * u2.T
    Bnew = torch.cat([
        torch.cat([F11inv, -u3], 1),
        torch.cat([-u3.T, d], 1)
    ], 0)
    return Bnew

add_new_col_to_inverse_new(X, torch.inverse(Nm1 - XtX), xnew, N=Nm1, n=nm1)

tensor([[-0.1303]])

In [127]:
nm1

tensor([0.7000])

In [163]:
def add_new_col_to_inverse_new(X, old_inverse, new_col, N=None, n=None):
    B = old_inverse
    v = new_col
    # Decomposition of diagonal matrix N
    Nlong = torch.cat([torch.sqrt(N), torch.zeros([X.shape[0] - N.shape[0], N.shape[1]])], 0)
    X = (Nlong - X)
    v = n - v
    u1 = torch.matmul(X.T, v)
    u2 = torch.matmul(B, u1)
    d = 1. / (torch.matmul(v.T, v) - torch.matmul(u1.T, u2))
    return n - d

print(add_new_col_to_inverse_new(X, torch.inverse(Nm1 - XtX), xnew, N=Nm1, n=nm1))

tensor([[5.2744]])


In [189]:
%timeit torch.inverse(Nm1 + torch.matmul(X.T, X))

15.2 µs ± 206 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [188]:
import numpy as np
def woodbury(A, U, V, k):
    A_inv = torch.diag(1. / torch.diag(A))  # Fast matrix inversion of a diagonal.
    B_inv = torch.inverse(torch.eye(k) + V @ A_inv @ U)
    return A_inv - (A_inv @ U @ B_inv @ V @ A_inv)

%timeit woodbury(Nm1, X.T, X, X.shape[0])

64.3 µs ± 1.47 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [182]:
# Can possible use Woodbury identity..
Am1 = torch.inverse(Nm1)
u = X.T
c = torch.eye(X.shape[0])
v = X

left = torch.mm(Am1, u)
middle = (c + torch.mm(torch.mm(v, Am1), u))
right = torch.mm(v, Am1)

Am1 - torch.mm(torch.mm(left, middle), right)

tensor([[ 0.1611, -0.0021, -0.0120],
        [-0.0021,  0.1594, -0.0018],
        [-0.0120, -0.0018,  0.1530]])

In [173]:
o = torch.mm(N1, X.T)
p = torch.eye(X.shape[0]) + torch.matmul(X, X.T)
o = torch.mm(o, p)
o = torch.mm(o, X)
N1 - o

tensor([[ 0.0475, -0.0217, -0.1195],
        [-0.0214,  0.0674, -0.0202],
        [-0.1180, -0.0203, -0.0151]])

In [167]:
torch.inverse(Nm1_new - XtXnew)

tensor([[0.1920, 0.0023, 0.0131, 0.0324],
        [0.0023, 0.1869, 0.0021, 0.0082],
        [0.0131, 0.0021, 0.1960, 0.0387],
        [0.0324, 0.0082, 0.0387, 1.5992]])

In [394]:
import torch
import time
import torch.nn.functional as F


n, p = 128, 4000
col = 0

a = torch.randn(100, 16) * 0.5
b = torch.randn(500, 16) * 0.5

match_ab = torch.matmul(a, torch.t(b))
match_ba = torch.t(match_ab)
match_bb = torch.matmul(b, torch.t(b))

p_ab = F.softmax(match_ab, dim=1)
p_bb = F.softmax(match_bb, dim=1)
p_ba = F.softmax(match_ba, dim=1)

In [395]:
match_ba_bb = torch.cat([match_ba, match_bb], dim=1)
p_ba_bb = torch.clamp(F.softmax(match_ba_bb, dim=1), min=1e-8)

In [396]:
N = a.shape[0]
M = b.shape[0]
Tbar_ul, Tbar_uu = p_ba_bb[:, :N], p_ba_bb[:, N:]

In [430]:
middle = torch.inverse(torch.eye(Tbar_uu.shape[1]) - Tbar_uu)# - torch.eye(Tbar_uu.shape[1])

In [433]:
middle = torch.eye(Tbar_uu.shape[1])
for i in range(1, 3):
    middle += torch.matrix_power(Tbar_uu, i)
middle /= Tbar_uu.sum(1, keepdim=True)

In [434]:
p_aba = torch.matmul(torch.matmul(p_ab, middle), Tbar_ul)

In [432]:
p_aba

tensor([[0.0115, 0.0085, 0.0108,  ..., 0.0093, 0.0088, 0.0090],
        [0.0093, 0.0101, 0.0103,  ..., 0.0093, 0.0085, 0.0092],
        [0.0094, 0.0082, 0.0150,  ..., 0.0095, 0.0084, 0.0092],
        ...,
        [0.0094, 0.0086, 0.0111,  ..., 0.0118, 0.0089, 0.0093],
        [0.0094, 0.0083, 0.0103,  ..., 0.0094, 0.0102, 0.0089],
        [0.0089, 0.0083, 0.0103,  ..., 0.0090, 0.0081, 0.0117]])

In [437]:
p_aba / p_aba.sum(1, keepdim=True)

tensor([[0.0151, 0.0088, 0.0112,  ..., 0.0096, 0.0092, 0.0085],
        [0.0094, 0.0128, 0.0100,  ..., 0.0097, 0.0083, 0.0090],
        [0.0097, 0.0081, 0.0217,  ..., 0.0103, 0.0081, 0.0089],
        ...,
        [0.0096, 0.0090, 0.0118,  ..., 0.0157, 0.0092, 0.0091],
        [0.0098, 0.0083, 0.0100,  ..., 0.0099, 0.0126, 0.0081],
        [0.0084, 0.0082, 0.0101,  ..., 0.0090, 0.0074, 0.0153]])

In [165]:
match_ab = torch.matmul(a, torch.t(b))
p_ab = F.softmax(match_ab, dim=1)

match_ba = torch.t(match_ab)
match_bb = torch.matmul(b, torch.t(b))
p_bb = torch.softmax(match_bb, dim=1)
p_ba = F.softmax(match_ba, dim=1)

match_ba_bb = torch.cat([match_ba, match_bb], dim=1)
p_ba_bb = torch.clamp(F.softmax(match_ba_bb, dim=1), min=1e-8)
N = a.shape[0]
M = b.shape[0]
Tbar_ul, Tbar_uu = p_ba_bb[:, :N], p_ba_bb[:, N:]
I = torch.eye(M)
I = I.cuda() if Tbar_uu.is_cuda else I
# middle = torch.inverse(I - Tbar_uu + 1e-8)
middle = torch.inverse(match_bb + 1e-8)
# middle = torch.inverse(I - p_bb + 1e-8)
p_aba = torch.matmul(torch.matmul(p_ab, middle), Tbar_ul)
p_aba /= p_aba.sum(1, keepdim=True)
# p_aba = torch.matmul(torch.matmul(p_ab, p_bb), p_ba)

In [162]:
p_aba.sum(1)

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000])

In [163]:
middle

tensor([[ 1.2201, -0.3730,  0.6471,  ..., -0.3973, -0.3934, -0.2369],
        [-2.3164,  0.3392, -4.5347,  ...,  0.9172,  2.8041, -0.4114],
        [ 3.0603,  0.5993,  3.7068,  ..., -0.6661, -5.3733,  2.5896],
        ...,
        [ 2.4616,  0.9285,  3.8963,  ..., -0.7809, -5.4913,  1.0392],
        [-1.4368, -0.3912, -3.5948,  ...,  0.0509,  3.4498, -0.8072],
        [-1.6878, -0.0854, -2.4238,  ...,  0.5706,  0.5472, -1.4204]])

In [128]:
p_aba

tensor([[6.0889, 6.0888, 6.0887,  ..., 6.0888, 6.0890, 6.0888],
        [6.0889, 6.0888, 6.0887,  ..., 6.0888, 6.0890, 6.0888],
        [6.0889, 6.0888, 6.0887,  ..., 6.0888, 6.0890, 6.0888],
        ...,
        [6.0889, 6.0888, 6.0887,  ..., 6.0888, 6.0890, 6.0888],
        [6.0889, 6.0888, 6.0887,  ..., 6.0888, 6.0890, 6.0888],
        [6.0889, 6.0888, 6.0887,  ..., 6.0888, 6.0890, 6.0888]])

In [123]:
p_bb

tensor([[0.0003, 0.0002, 0.0003,  ..., 0.0003, 0.0002, 0.0002],
        [0.0002, 0.0003, 0.0003,  ..., 0.0002, 0.0003, 0.0003],
        [0.0003, 0.0003, 0.0003,  ..., 0.0002, 0.0002, 0.0003],
        ...,
        [0.0003, 0.0002, 0.0002,  ..., 0.0003, 0.0002, 0.0003],
        [0.0002, 0.0003, 0.0002,  ..., 0.0002, 0.0003, 0.0002],
        [0.0002, 0.0003, 0.0003,  ..., 0.0003, 0.0002, 0.0003]])

In [ ]:
p_ab

In [113]:
middle

tensor([[1.0100, 0.0100, 0.0100,  ..., 0.0100, 0.0100, 0.0100],
        [0.0100, 1.0100, 0.0100,  ..., 0.0100, 0.0100, 0.0100],
        [0.0100, 0.0100, 1.0100,  ..., 0.0100, 0.0100, 0.0100],
        ...,
        [0.0100, 0.0100, 0.0100,  ..., 1.0100, 0.0100, 0.0100],
        [0.0100, 0.0100, 0.0100,  ..., 0.0100, 1.0100, 0.0100],
        [0.0100, 0.0100, 0.0100,  ..., 0.0100, 0.0100, 1.0100]])

In [114]:
Tbar_ul

tensor([[0.0002, 0.0002, 0.0002,  ..., 0.0002, 0.0002, 0.0002],
        [0.0002, 0.0002, 0.0002,  ..., 0.0002, 0.0002, 0.0002],
        [0.0002, 0.0002, 0.0002,  ..., 0.0002, 0.0002, 0.0002],
        ...,
        [0.0002, 0.0002, 0.0002,  ..., 0.0002, 0.0002, 0.0002],
        [0.0002, 0.0002, 0.0002,  ..., 0.0002, 0.0002, 0.0002],
        [0.0002, 0.0002, 0.0002,  ..., 0.0002, 0.0002, 0.0002]])

In [115]:
p_aba

tensor([[0.0100, 0.0100, 0.0100,  ..., 0.0100, 0.0100, 0.0100],
        [0.0100, 0.0100, 0.0100,  ..., 0.0100, 0.0100, 0.0100],
        [0.0100, 0.0100, 0.0100,  ..., 0.0100, 0.0100, 0.0100],
        ...,
        [0.0100, 0.0100, 0.0100,  ..., 0.0100, 0.0100, 0.0100],
        [0.0100, 0.0100, 0.0100,  ..., 0.0100, 0.0100, 0.0100],
        [0.0100, 0.0100, 0.0100,  ..., 0.0100, 0.0100, 0.0100]])

In [96]:
(p_aba / p_aba.sum(1, keepdim=True))

tensor([[0.5052, 0.4948],
        [0.5018, 0.4982]])

In [55]:
match_ab = torch.matmul(a, torch.t(b))
p_ab = F.softmax(match_ab, dim=1)
p_ba = F.softmax(torch.t(match_ab), dim=1)
p_aba = torch.matmul(p_ab, p_ba)

In [56]:
p_aba.sum(1)

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000])

In [139]:
X = torch.randn([n, p]) * 0.5
# X = torch.softmax(X, 0)
B = torch.matmul(X.T, X)
B = B.cuda()
B = B.cpu()
now = time.time()
B = torch.inverse(B)
time.time() - now

NameError: name 'n' is not defined

In [7]:
new_col = v = torch.randn([n, 1]) * 0.5
# new_col = v = torch.softmax(torch.randn([n, 1]) * 0.5, 0)

In [8]:
def add_new_col_to_inverse(X, old_inverse, new_col):
    B = old_inverse
    v = new_col
    
    u1 = torch.matmul(X.T, v)
    u2 = torch.matmul(B, u1)
    F22inv = 1. / (torch.matmul(v.T, v) - torch.matmul(u1.T, u2))
    u3 = F22inv * u2
    F11inv = B + F22inv * u2 * u2.T
    Bnew = torch.cat([
        torch.cat([F11inv, -u3], 1),
        torch.cat([-u3.T, F22inv], 1)
    ], 0)
    return Bnew

In [9]:
now = time.time()
add_new_col_to_inverse(B, v)
time.time() - now

0.338165283203125

In [29]:
Xnew = torch.cat([X, v], 1)
torch.inverse(torch.matmul(Xnew.T, Xnew))

tensor([[ 1.2506, -0.6256,  0.1035,  1.0205],
        [-0.6256,  0.8223,  0.1486, -0.5141],
        [ 0.1035,  0.1486,  0.4402,  0.3598],
        [ 1.0205, -0.5141,  0.3598,  2.0012]])